In [1]:
import os #여기부터 실행하세요. one drive에서 파일 다운받아두세요. 
import re
import openpyxl
import calendar
from bs4 import BeautifulSoup
from pathos.multiprocessing import ProcessingPool as Pool
import pandas as pd
import time

cal = calendar.Calendar()

query = "여성"
query2 = "#2_" + query
year = 2019
xlist = list()

for month in range(1, 13):
        monthdays = [d for d in cal.itermonthdays(year, month) if d != 0]
        for day in monthdays:
            dd = str(year) + str(month).zfill(2) + str(day).zfill(2)
            xlsx_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#2_html2text', query2, '#2_'+ str(year) + '_'+query, 
                                     '{}.xlsx'.format("#2_"+dd+"_"+query))
            xlist.append(xlsx_name)

In [239]:
def replace_sub(text):
                
        from nltk.tokenize import regexp_tokenize
        from string import punctuation
        
        tlist, ttlist= list(), list()
        text = regexp_tokenize(text, r'[^\s\.][^\.\n]+')
        for _ in text:
            if len(_) > 11:
                tlist.append(_)
                
        text = ".\n".join(tlist) + "."

        text = re.sub(r'\d+\.(\n)\d+', r'', text)
        text = re.sub(r'\b(\w|[.])+@(?:[.]?\w+)+\b', ' ', text)
        text = re.sub(r'\bhttps?://\w+(?:[.]?\w+)+\b', ' ', text)
        text = re.sub(r'[^A-Za-z0-9가-힣ㄱ-하-ㅣ\.?!\n ]', ' ', text)
        text = re.sub(r'\b[a-z][A-Za-z0-9]+\b', ' ', text)
        text = re.sub(r'\s{2,}', ' ', text) 
        text = re.sub(r'[\(\[].*?[\)\]]', '', text)
        text = re.sub('【.*】','',text)
        text = re.sub(r'"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\/:;<=>\@\\[\\\\\\]\\^_`\\{\\|\\}\\~', '', text)
        
        # 여기부터 새로운 태그 추가 #신문사 이름은 나중에 한꺼번에 뺄 것이니 추가하지 말 것
        text = re.sub(r"-\d+-", '', text) # 이거 전에 '-'제거하면 안됨.
        text = re.sub(r"\b\w{3}\s기자", '', text)
        text = re.sub(r'\s?[a-z]\s', ' ', text)
        text = re.sub(r'\s+\d+px?', '', text)
        text = re.sub(r'\s\d+\s\d+\s\d+\s', '', text)
        text = re.sub(r'\s?(\d+\s)', '', text)
        text = re.sub(r'((\d+[a-z])+)', '', text)
        text = re.sub(r'[A-Z0-9a-z][A-Z0-9a-z]', '', text)
        text = re.sub(r'\b[A-Z0-9]\b', '', text)

        
        text = text.replace('출처', '').replace('방송캡처', '').replace('-', '').replace('오류를 우회하기 위한 함수 추가.', '')
        text = text.replace('동영상 뉴스.', '').replace('아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다.', '')
        text = text.replace(' 사진 ', ' ').replace('경남신문의 콘텐츠는 저작권법의 보호를 받는 바 무단전재 복사 재배포를 금합니다.', '')
        text = text.replace('돋움', '').replace('메인이미지', '').replace('. p .', "\n").replace('정상처리되지 않았습니다.', '')
        text = text.replace(' p ', '\n').replace('IMGFLOATING ', '').replace('캡쳐 ', '').replace(' P ', '').replace('INSERT RANDOM NUMBER HERE', '')
        text = text.replace('OptimaA Scrip Ta', '').replace('시원하게 털어드립니다.', '').replace('저작권자 인사이트 무단전재 및 재배포 금지 .', '')
        text = text.replace('수직배너 와이드', '').replace('www.', '').replace('동영상이 안 나오면', '').replace('여기를 누르시면 동영상을 별도의 새 창에서 볼 수 있습니다.', '')
        try: 
            text = text[:text.rindex('다.')+2]
            
            text = text.split(sep='.')
            for _ in text:
                if len(re.sub(r'\s{2,}', ' ', _)) > 11:
                    ttlist.append(re.sub(r'\s{2,}', ' ', _))
                    
            text = ".".join(ttlist) + "."
            
            return text
        except:
            return 0


In [263]:
"""
유미 0-91 >>>>>>>>>> 0,6
지은 92-182 >>>>>>>>>>
은서 183-273 >>>>>>>>>>
유리 274-364 >>>>>>>>>>
"""

df = pd.read_excel(xlist[0]) # 여기에 숫자 바꿔가면서 하면 돼

n = 93
text = df.iloc[n, 3]
url = df.iloc[n, 4]
print(url)
print(replace_sub(text))
#re.findall(r'((\d+[a-z])+)', text)

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0010556680

경찰대 치안정책연구소 전망 보이스피싱 불법사금융 위험 경고 연합뉴스 제공 서울 연합뉴스 정부가 가계대출을 규제할 여러 대책을 내놓은 가운데 대출이 필요하거나 이자 부담에 시달리는 서민들을 상대로 한 지능범죄가 올해에도 기승을 부릴 것이라는 전망이 나왔다.
경찰대 치안정책연구소는 1일 발간한 치안전망보고서에서 보이스피싱 전화금융사기 이나 불법 사금융 범죄 피해자를 양산하는 환경적 요인이 올해에도 쉽게 개선될 것으로 보이지 않는다 며 이런 범죄가 계속해서 증가할 것으로 보인다고 밝혔다.
연구소는 지난해 보이스피싱과 불법 사금융 범죄 통계를 살펴본 결과 대출이 필요하지만 제도권의 대출 서비스를 이용하지 못하거나 대출이자 압박을 느끼는 서민들의 경제상황과 이들 범죄 양상 간 관련성이 발견됐다고 지적했다.
농협 금감원 경찰청 보이스피싱 꺼져! 서울 연합뉴스 지난해 월5일 오전 서울역 앞에서 열린 보이스피싱 제로 캠페인에서 농협 금감원 경찰청 임직원들이 퍼포먼스를 하고 있다.
이들은 캠페인을 통해 시민들에게 수사기관을 사칭해 개인정보가 유출되었다는 보이스피싱 전화를 조심하고 피해 발생시경찰청 과금융감독원 및 해당금융회사에 신고할 것을 당부했다.
실제 금융감독원 통계를 보면 작년 상반기 보이스피싱 유형 가운데 대출빙자형 피해액이 1천4억원으로 전체 1천2억원 의 를 차지하며 정부기관 등 사칭형 피해액 8억원 을 크게 웃돌았다.
경찰청의 불법 사금융 통계에서도 불법 채권추심 유사수신 불법 다단계 등 다른 유형은 모두 전년보다 검거 건수가 큰 폭으로 줄었으나 불법 대부업만증가했다.
연구소는 이들 통계에 대해 공통적으로 가계부채 1천0조라는 우리나라의 현 실정을 반영하는 동시에 수많은 서민들이 범죄 피해자로 노출되는 원인을 알려준다 며 수많은 서민이 대출을 필요로 하지만 제도권에서 대출을 받지 못하거나 이자 압박을 심각하게 

In [47]:
import requests
from bs4 import BeautifulSoup

tag = "div.cont_cont"

raw = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
html = BeautifulSoup(raw.text, "html.parser")
html.select(tag)

[<div class="cont_cont"><ul> <li class="blank30"></li><li id="content_li"><div class="ad_content_right"><ins class="adsbygoogle" data-ad-client="ca-pub-2748737563670298" data-ad-slot="6519210004" style="display:inline-block;width:200px;height:200px"></ins>
 <script>(adsbygoogle = window.adsbygoogle || []).push({});</script>
 </div><p>
 <br/> 2019년 경남에서는 소상공인 카드수수료 절감 정책인 ‘제로페이’가 본격 운영되고, 초·중·고교 모든 학생에게 무상급식이 제공된다. 신혼부부 생애최초 주택에 대해서는 지방세가 감면되고, 저소득층 노인 임플란트 시술 본인부담금도 지원된다. 경남도는 이 같은 내용을 담은 7개 분야 100개 시책과 제도를 발표했다.<br/>
 <br/>
 <div align="center" id="content_imgbox">
 <div class="photo_content" style="padding: 10px 10px 3px; border: 1px solid rgb(255, 255, 255); border-image: none; width: 600px; text-align: left; color: rgb(102, 102, 102); font-family: 돋움,dotum; font-size: 12px; margin-bottom: 15px;">
 <img alt="메인이미지" src="/edb/nimages/2018/12/20181227.01010113000001.01L.jpg" style="width: 100%; margin-bottom: 4px;"/>
 <br/>
  지난 12월 26일 창원시 가로수길 상점에서 김경수 도지사가 직접 제로페이 결제를 선보이고 있다. 이날 김